<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/StefanMaxwell_NRTL_JAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
import jax.numpy as jnp

#### Using the Non-Random Two Liquid (NRTL) activity coefficient, one can calculate the activity coefficient of a component $\gamma_i$ in a mixture by:

$$\ln{\gamma_i}=\dfrac{\sum\limits_{j=1}^n x_j \tau_{ji} G_{ji}}{\sum\limits_{k=1}^n {x_k G_{ki}}} + 
\sum\limits_{j=1}^n 
\dfrac {x_j G_{ij}}{\sum\limits_{k=1}^{n}x_k G_{kj}}
\left(\tau_{ij}-
\dfrac{\sum\limits_{m=1}^n x_m \tau_{mj} G_{mj}}
{\sum\limits_{k=1}^n x_k G_{kj}}
\right)
$$

$$\begin{align}
G_{ij} &= \exp \left(-\alpha_{ij}\tau_{ij} \right) \\
\tau_{ij} &= A_{ij} + \dfrac{B_{ij}}{T} + C_{ij}\ln{T} + D_{ij}T \\
\alpha_{ij} &= \alpha_{ji}
\end{align}$$


here, $x$ is a vector of mole fractions while $\tau$ and $G$ are matrices calculated from $A$, $B$, $C$, $D$, and $\alpha$ that are matrices of binary interaction parameters determined experimentally.  Unlike the other matrices, the $\alpha$ interaction matrix is symmetric.

In [2]:
# For the 3 component mixture: (1) Methanol, (2) Ethanol, and (3) Water system, 
# the matrices A, C, D are zeros, while B and alpha have been determined experimentally.

B=jnp.array( [[   0.,   -155.5,   -24.49],
 [ 190.06,    0.,    -55.17],
 [ 307.17,  670.44,    0.  ]])

alpha = jnp.array( [[0.,    0.305, 0.3  ],
 [0.305, 0.,    0.303],
 [0.3,   0.303, 0.   ]])

In [10]:
def ln_g(xm1, T):
    x=jnp.asarray(jnp.append(xm1,1-jnp.sum(xm1))).reshape((1,-1))
    tau = B/T
    G = jnp.exp(-alpha * tau)
    xG=x @ G
    xtauGdivxG = (x@ (tau*G)/ xG)
    lngamma = xtauGdivxG +  (((G*(tau - xtauGdivxG))/xG) @x.T).T
    return lngamma.flatten()

In [19]:
x12=jnp.array([0.2, 0.3])
T=300
print(jnp.exp(ln_g(x12,T)))
dlng_dx=jax.jacobian(ln_g)
dlng_dx(x12,T)

[1.0061771 1.335594  1.4216974]


DeviceArray([[ 0.04480401, -0.65847015],
             [-1.1157057 , -1.4089153 ],
             [ 0.6515017 ,  1.108737  ]], dtype=float32)

In [16]:
#thermodynamic factor matrix (Eq 1.31 and 1.32 - Principles and Modern Applications of Mass Transfer Operations
#2nd Edition, Benitez)
tfm = jnp.eye(2) + x12[None,:]*dlng_dx(x12,T)[:2,:]
tfm

DeviceArray([[ 1.0089608 , -0.19754106],
             [-0.22314115,  0.5773254 ]], dtype=float32)

In [17]:
#driving force
dxdz = jnp.array([0.3, 0.4]).reshape((-1,1))
d = tfm @ dxdz
d

DeviceArray([[0.22367185],
             [0.16398782]], dtype=float32)